### FINAL RESULTS:
This data was taken from:https://www.kaggle.com/utkarshxy/who-worldhealth-statistics-2020-complete?select=incedenceOfMalaria.csv.

It is a complete set of recent and updated health statistics of the world. It covers many different countries and years. The Kaggle link contains many different CSV files with each covering a specific statistic. The cleaning done here attempts to merge all the CSVs into a single usable dataframe.

The steps taken to clean the data are as follows:
1. Aggregate repeated variables
2. Group by Country and Year
2. Rename Columns
3. Drop CSVs with Interval Years
4. eliminate records from before 2008
3. Merge Dataframes
4. Drop columns with significant amount of NaNs

There remains a few issues with the data:
1. For some countries and statistics, we do not have every year in the interval 2008-2019
2. Very few columns are kept after all of the data cleaning







In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pandas_profiling import ProfileReport

In [62]:
def clean_df(df):
#     rename the columns
    column_name = df["Indicator"].iloc[0]
    if "Dim1" in df.columns:
        df =df.groupby("Period").first()
    df.reset_index(inplace=True)
    df = df[["Location",  "First Tooltip", "Period"]]
    df.rename(columns = {"First Tooltip": column_name}, inplace=True)
    return df


In [ ]:
dir_list = os.listdir("health/")
master_df = None
for idx, csv_name in enumerate(dir_list):
    if csv_name.endswith('csv'):
        curr_df = pd.read_csv("health/" + csv_name)
        curr_df = clean_df(curr_df)
#         merge df
        if idx == 1:
            master_df = curr_df
        else:
            if curr_df["Period"].dtype == "int64":
                curr_df["Period"] = curr_df["Period"].astype(int)
                master_df = master_df.merge(curr_df, on=["Location", "Period"], how="outer")

In [ ]:
df = master_df[master_df['Period']>=2008]
df['Reported number of people requiring interventions against NTDs'] = df['Reported number of people requiring interventions against NTDs'].fillna(-1).astype(int)
z = df.count(axis=0)
df.to_csv('merged.csv', index = False)
df.drop(labels=df.columns[z<len(df)//2], axis=1, inplace = True)
df.to_csv('merged_cleaned.csv', index = False)
df.head()